In [1]:
import pandas as pd # (version pandas 1.4.2)
import numpy as np  # (version numpy 1.22.4)
import seaborn as sns # (version seaborn 0.11.2)
import scipy.stats as st
import datetime 
sns.set_palette("colorblind")

import matplotlib.pyplot as plt
import plotly.graph_objects as go
#import lux

from scipy.stats import shapiro

colors = ["#b2eca7","#ffcc97","#676283","#72bbff","#d2a0f2","#0abdc0"]

# Pour éviter d'avoir les messages warning
import warnings
warnings.filterwarnings('ignore')

In [2]:
df_region_country = pd.read_csv('DAN-P8-donnees/RegionCountry.csv') 
df_population = pd.read_csv('DAN-P8-donnees/Population.csv') 
df_political = pd.read_csv('DAN-P8-donnees/PoliticalStability.csv') 
df_motalite = pd.read_csv('DAN-P8-donnees/MortalityRateAttributedToWater.csv') 
df_safe = pd.read_csv('DAN-P8-donnees/BasicAndSafelyManagedDrinkingWaterServices.csv') 

#df_stabilite = pd.read_excel('stabilité_data.xlsx')

In [3]:
df_motalite['WASH deaths'] = df_motalite['WASH deaths']* 1000

In [4]:
df_motalite.head()

,Year,Country,Granularity,Mortality rate attributed to exposure to unsafe WASH services,WASH deaths
0,2016,Afghanistan,Female,15.31193,NaN
1,2016,Afghanistan,Male,12.61297,NaN
2,2016,Afghanistan,Total,13.92067,4824353.0
3,2016,Albania,Female,0.12552,NaN
4,2016,Albania,Male,0.20650,NaN


In [5]:
df_political.Country[df_political.Country == 'China, mainland'] = 'China'

In [6]:
df_population['Country'].nunique()

239

In [7]:
chine3 = df_political.loc[df_political['Country'] == "China, mainland",:]
chine3

,Country,Year,Political_Stability,Granularity


In [8]:
df_region_country.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194 entries, 0 to 193
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   REGION (DISPLAY)   194 non-null    object
 1   COUNTRY (DISPLAY)  194 non-null    object
dtypes: object(2)
memory usage: 3.2+ KB


In [9]:
#On stock les deux lignes contenant des informations dans chaques colonnes sauf 'sku' dans deux dataframe
chine = df_region_country.loc[df_region_country['COUNTRY (DISPLAY)'] == "China",:]
chine

,REGION (DISPLAY),COUNTRY (DISPLAY)
18,Western Pacific,China


In [10]:
df_motalite.head()

,Year,Country,Granularity,Mortality rate attributed to exposure to unsafe WASH services,WASH deaths
0,2016,Afghanistan,Female,15.31193,NaN
1,2016,Afghanistan,Male,12.61297,NaN
2,2016,Afghanistan,Total,13.92067,4824353.0
3,2016,Albania,Female,0.12552,NaN
4,2016,Albania,Male,0.20650,NaN


In [11]:
df_population.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20914 entries, 0 to 20913
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Country      20914 non-null  object 
 1   Granularity  20914 non-null  object 
 2   Year         20914 non-null  int64  
 3   Population   20914 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 653.7+ KB


In [12]:
df_population.head()

,Country,Granularity,Year,Population
0,Afghanistan,Total,2000,20779.953
1,Afghanistan,Male,2000,10689.508
2,Afghanistan,Female,2000,10090.449
3,Afghanistan,Rural,2000,15657.474
4,Afghanistan,Urban,2000,4436.282


In [13]:
df_population['Population'] = df_population['Population']* 1000

In [14]:
df_population.head()

,Country,Granularity,Year,Population
0,Afghanistan,Total,2000,20779953.0
1,Afghanistan,Male,2000,10689508.0
2,Afghanistan,Female,2000,10090449.0
3,Afghanistan,Rural,2000,15657474.0
4,Afghanistan,Urban,2000,4436282.0


In [15]:
#pop = df_population.pivot(index=['Country','Year'], columns='Granularity', values='Population').reset_index()

#pop['% Urban'] = (pop['Urban']/pop['Total'])*100
#pop.to_csv("P8_Population_Pivot.csv")
#pop

In [16]:
df_population['Country'].nunique()

239

In [17]:
df_final_tmp = pd.merge(df_region_country, df_population, left_on='COUNTRY (DISPLAY)', right_on='Country', how='right', indicator=True)

In [18]:
df_final_tmp['Country'].nunique()

239

In [19]:
df_final_tmp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20914 entries, 0 to 20913
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   REGION (DISPLAY)   17737 non-null  object  
 1   COUNTRY (DISPLAY)  17737 non-null  object  
 2   Country            20914 non-null  object  
 3   Granularity        20914 non-null  object  
 4   Year               20914 non-null  int64   
 5   Population         20914 non-null  float64 
 6   _merge             20914 non-null  category
dtypes: category(1), float64(1), int64(1), object(4)
memory usage: 1.1+ MB


In [20]:
df_final_tmp.head()

,REGION (DISPLAY),COUNTRY (DISPLAY),Country,Granularity,Year,Population,_merge
0,Eastern Mediterranean,Afghanistan,Afghanistan,Total,2000,20779953.0,both
1,Eastern Mediterranean,Afghanistan,Afghanistan,Male,2000,10689508.0,both
2,Eastern Mediterranean,Afghanistan,Afghanistan,Female,2000,10090449.0,both
3,Eastern Mediterranean,Afghanistan,Afghanistan,Rural,2000,15657474.0,both
4,Eastern Mediterranean,Afghanistan,Afghanistan,Urban,2000,4436282.0,both


In [21]:
# On regarde si la colonne _merge contient que des 'both'
df_test = df_final_tmp[df_final_tmp['_merge']!='both']

print(df_test.shape)
df_test.head()

(3177, 7)


,REGION (DISPLAY),COUNTRY (DISPLAY),Country,Granularity,Year,Population,_merge
285,NaN,NaN,American Samoa,Total,2000,57821.0,right_only
286,NaN,NaN,American Samoa,Rural,2000,6565.0,right_only
287,NaN,NaN,American Samoa,Urban,2000,50956.0,right_only
288,NaN,NaN,American Samoa,Total,2001,58494.0,right_only
289,NaN,NaN,American Samoa,Rural,2001,6695.0,right_only


In [22]:
# On regarde si la colonne _merge contient que des 'both'
df_test2 = df_final_tmp[(df_final_tmp['_merge']!='both') & (df_final_tmp['Country']!='American Samoa')]

print(df_test2.shape)
df_test2.head()

(3120, 7)


,REGION (DISPLAY),COUNTRY (DISPLAY),Country,Granularity,Year,Population,_merge
494,NaN,NaN,Anguilla,Total,2000,11252.0,right_only
495,NaN,NaN,Anguilla,Rural,2000,0.0,right_only
496,NaN,NaN,Anguilla,Urban,2000,11070.0,right_only
497,NaN,NaN,Anguilla,Total,2001,11515.0,right_only
498,NaN,NaN,Anguilla,Rural,2001,0.0,right_only


In [23]:
df_motalite.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 549 entries, 0 to 548
Data columns (total 5 columns):
 #   Column                                                         Non-Null Count  Dtype  
---  ------                                                         --------------  -----  
 0   Year                                                           549 non-null    int64  
 1   Country                                                        549 non-null    object 
 2   Granularity                                                    549 non-null    object 
 3   Mortality rate attributed to exposure to unsafe WASH services  549 non-null    float64
 4   WASH deaths                                                    183 non-null    float64
dtypes: float64(2), int64(1), object(2)
memory usage: 21.6+ KB


In [24]:
df_motalite

,Year,Country,Granularity,Mortality rate attributed to exposure to unsafe WASH services,WASH deaths
0,2016,Afghanistan,Female,15.31193,NaN
1,2016,Afghanistan,Male,12.61297,NaN
2,2016,Afghanistan,Total,13.92067,4824353.0
3,2016,Albania,Female,0.12552,NaN
4,2016,Albania,Male,0.20650,NaN
...,...,...,...,...,...
544,2016,Zambia,Male,36.61913,NaN
545,2016,Zambia,Total,34.91273,5792504.0
546,2016,Zimbabwe,Female,22.16388,NaN
547,2016,Zimbabwe,Male,27.06688,NaN


In [25]:
df_safe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10476 entries, 0 to 10475
Data columns (total 5 columns):
 #   Column                                                       Non-Null Count  Dtype  
---  ------                                                       --------------  -----  
 0   Year                                                         10476 non-null  int64  
 1   Country                                                      10476 non-null  object 
 2   Granularity                                                  10476 non-null  object 
 3   Population using at least basic drinking-water services (%)  9415 non-null   float64
 4   Population using safely managed drinking-water services (%)  3286 non-null   float64
dtypes: float64(2), int64(1), object(2)
memory usage: 409.3+ KB


In [26]:
df_political.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3526 entries, 0 to 3525
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Country              3526 non-null   object 
 1   Year                 3526 non-null   int64  
 2   Political_Stability  3526 non-null   float64
 3   Granularity          3526 non-null   object 
dtypes: float64(1), int64(1), object(2)
memory usage: 110.3+ KB


In [27]:
# On supprime la colonne '_merge'
df_final_tmp = df_final_tmp.drop(['_merge'], axis=1)

In [28]:
# On supprime la colonne '_merge'
df_final_tmp = df_final_tmp.drop(['COUNTRY (DISPLAY)'], axis=1)

In [29]:
df_final_tmp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20914 entries, 0 to 20913
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   REGION (DISPLAY)  17737 non-null  object 
 1   Country           20914 non-null  object 
 2   Granularity       20914 non-null  object 
 3   Year              20914 non-null  int64  
 4   Population        20914 non-null  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 980.3+ KB


In [30]:
chine3 = df_safe.loc[df_safe['Country'] == "China, mainland",:]
chine3

,Year,Country,Granularity,Population using at least basic drinking-water services (%),Population using safely managed drinking-water services (%)


In [31]:
df_final_tmp1 = pd.merge(df_final_tmp, df_safe, on=['Country', 'Year', 'Granularity'], how='outer', indicator=True)

In [32]:
# On supprime la colonne '_merge'
df_final_tmp1 = df_final_tmp1.drop(['_merge'], axis=1)

In [33]:
df_final_tmp1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21076 entries, 0 to 21075
Data columns (total 7 columns):
 #   Column                                                       Non-Null Count  Dtype  
---  ------                                                       --------------  -----  
 0   REGION (DISPLAY)                                             17737 non-null  object 
 1   Country                                                      21076 non-null  object 
 2   Granularity                                                  21076 non-null  object 
 3   Year                                                         21076 non-null  int64  
 4   Population                                                   20914 non-null  float64
 5   Population using at least basic drinking-water services (%)  9415 non-null   float64
 6   Population using safely managed drinking-water services (%)  3286 non-null   float64
dtypes: float64(3), int64(1), object(3)
memory usage: 1.3+ MB


In [34]:
df_final_tmp2 = pd.merge(df_final_tmp1, df_motalite, on=['Country', 'Year','Granularity'], how='outer', indicator=True)

In [35]:
df_final_tmp2

,REGION (DISPLAY),Country,Granularity,Year,Population,Population using at least basic drinking-water services (%),Population using safely managed drinking-water services (%),Mortality rate attributed to exposure to unsafe WASH services,WASH deaths,_merge
0,Eastern Mediterranean,Afghanistan,Total,2000,20779953.0,27.77190,NaN,NaN,NaN,left_only
1,Eastern Mediterranean,Afghanistan,Male,2000,10689508.0,NaN,NaN,NaN,NaN,left_only
2,Eastern Mediterranean,Afghanistan,Female,2000,10090449.0,NaN,NaN,NaN,NaN,left_only
3,Eastern Mediterranean,Afghanistan,Rural,2000,15657474.0,21.61913,NaN,NaN,NaN,left_only
4,Eastern Mediterranean,Afghanistan,Urban,2000,4436282.0,49.48745,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...
21073,NaN,Republic of North Macedonia,Rural,2017,NaN,96.63789,68.79372,NaN,NaN,left_only
21074,NaN,Republic of North Macedonia,Total,2017,NaN,93.14146,81.00796,NaN,NaN,left_only
21075,NaN,Republic of North Macedonia,Urban,2017,NaN,90.58327,89.94465,NaN,NaN,left_only
21076,NaN,Republic of North Macedonia,Female,2016,NaN,NaN,NaN,0.05756,NaN,right_only


In [36]:
# On supprime la colonne '_merge'
df_final_tmp2 = df_final_tmp2.drop(['_merge'], axis=1)

In [37]:
df_final_tmp3 = pd.merge(df_final_tmp2, df_political, on=['Country', 'Year', 'Granularity'], how='outer', indicator=True)

In [38]:
df_final_tmp3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21078 entries, 0 to 21077
Data columns (total 11 columns):
 #   Column                                                         Non-Null Count  Dtype   
---  ------                                                         --------------  -----   
 0   REGION (DISPLAY)                                               17737 non-null  object  
 1   Country                                                        21078 non-null  object  
 2   Granularity                                                    21078 non-null  object  
 3   Year                                                           21078 non-null  int64   
 4   Population                                                     20914 non-null  float64 
 5   Population using at least basic drinking-water services (%)    9415 non-null   float64 
 6   Population using safely managed drinking-water services (%)    3286 non-null   float64 
 7   Mortality rate attributed to exposure to unsafe W

In [39]:
df_final_tmp3 = df_final_tmp3.rename(columns={'REGION (DISPLAY)': 'Region'})

In [40]:
df_final_tmp3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21078 entries, 0 to 21077
Data columns (total 11 columns):
 #   Column                                                         Non-Null Count  Dtype   
---  ------                                                         --------------  -----   
 0   Region                                                         17737 non-null  object  
 1   Country                                                        21078 non-null  object  
 2   Granularity                                                    21078 non-null  object  
 3   Year                                                           21078 non-null  int64   
 4   Population                                                     20914 non-null  float64 
 5   Population using at least basic drinking-water services (%)    9415 non-null   float64 
 6   Population using safely managed drinking-water services (%)    3286 non-null   float64 
 7   Mortality rate attributed to exposure to unsafe W

In [41]:
# On supprime la colonne '_merge'
df_final = df_final_tmp3.drop(['_merge'], axis=1)

In [42]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21078 entries, 0 to 21077
Data columns (total 10 columns):
 #   Column                                                         Non-Null Count  Dtype  
---  ------                                                         --------------  -----  
 0   Region                                                         17737 non-null  object 
 1   Country                                                        21078 non-null  object 
 2   Granularity                                                    21078 non-null  object 
 3   Year                                                           21078 non-null  int64  
 4   Population                                                     20914 non-null  float64
 5   Population using at least basic drinking-water services (%)    9415 non-null   float64
 6   Population using safely managed drinking-water services (%)    3286 non-null   float64
 7   Mortality rate attributed to exposure to unsafe WASH servi

In [43]:
df_final.to_csv('df_final.csv', index=False) 

In [44]:
df_final.duplicated().sum()

0

In [45]:
df_final.head(20)

,Region,Country,Granularity,Year,Population,Population using at least basic drinking-water services (%),Population using safely managed drinking-water services (%),Mortality rate attributed to exposure to unsafe WASH services,WASH deaths,Political_Stability
0,Eastern Mediterranean,Afghanistan,Total,2000,20779953.0,27.77190,NaN,NaN,NaN,-2.44
1,Eastern Mediterranean,Afghanistan,Male,2000,10689508.0,NaN,NaN,NaN,NaN,NaN
2,Eastern Mediterranean,Afghanistan,Female,2000,10090449.0,NaN,NaN,NaN,NaN,NaN
3,Eastern Mediterranean,Afghanistan,Rural,2000,15657474.0,21.61913,NaN,NaN,NaN,NaN
4,Eastern Mediterranean,Afghanistan,Urban,2000,4436282.0,49.48745,NaN,NaN,NaN,NaN
5,Eastern Mediterranean,Afghanistan,Total,2001,21606988.0,27.79726,NaN,NaN,NaN,NaN
6,Eastern Mediterranean,Afghanistan,Male,2001,11117754.0,NaN,NaN,NaN,NaN,NaN
7,Eastern Mediterranean,Afghanistan,Female,2001,10489238.0,NaN,NaN,NaN,NaN,NaN
8,Eastern Mediterranean,Afghanistan,Rural,2001,16318324.0,21.61913,NaN,NaN,NaN,NaN
9,Eastern Mediterranean,Afghanistan,Urban,2001,4648139.0,49.48745,NaN,NaN,NaN,NaN


In [46]:
df_final.nunique()

Region                                                               6
Country                                                            240
Granularity                                                          5
Year                                                                19
Population                                                       20515
Population using at least basic drinking-water services (%)       7704
Population using safely managed drinking-water services (%)       2877
Mortality rate attributed to exposure to unsafe WASH services      548
WASH deaths                                                        183
Political_Stability                                                444
dtype: int64

In [47]:
df_final['Country'].nunique()

240

In [48]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21078 entries, 0 to 21077
Data columns (total 10 columns):
 #   Column                                                         Non-Null Count  Dtype  
---  ------                                                         --------------  -----  
 0   Region                                                         17737 non-null  object 
 1   Country                                                        21078 non-null  object 
 2   Granularity                                                    21078 non-null  object 
 3   Year                                                           21078 non-null  int64  
 4   Population                                                     20914 non-null  float64
 5   Population using at least basic drinking-water services (%)    9415 non-null   float64
 6   Population using safely managed drinking-water services (%)    3286 non-null   float64
 7   Mortality rate attributed to exposure to unsafe WASH servi